In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import pandas as pd

In [13]:
train = pd.read_csv('../data/Train_dataset.csv')
train.head(1)

,Registration Number,Annual Turnover,Cuisine,City,Restaurant Location,Opening Day of Restaurant,Facebook Popularity Quotient,Endorsed By,Instagram Popularity Quotient,Fire Audit,Liquor License Obtained,Situated in a Multi Complex,Dedicated Parking,Open Sitting Available,Resturant Tier,Restaurant Type,Restaurant Theme,Restaurant Zomato Rating,Restaurant City Tier,Order Wait Time,Staff Responsivness,Value for Money,Hygiene Rating,Food Rating,Overall Restaurant Rating,Live Music Rating,Comedy Gigs Rating,Value Deals Rating,Live Sports Rating,Ambience,Lively,Service,Comfortablility,Privacy
0,60001,42000000,"indian,irish",Bangalore,Near Business Hub,14/02/09,84.3,Not Specific,95.8,1,1,1,0,1,2.0,Bar,Arabian,3,0,2,5,5,5,7,10.0,4.0,NaN,NaN,NaN,8.0,8,6,6,6


In [14]:
from auto_ml_kinder import pre_processing as pp

In [15]:
train.columns = train.columns.str.lower().str.replace(' ', '_')

In [16]:
train.head(1)

,registration_number,annual_turnover,cuisine,city,restaurant_location,opening_day_of_restaurant,facebook_popularity_quotient,endorsed_by,instagram_popularity_quotient,fire_audit,liquor_license_obtained,situated_in_a_multi_complex,dedicated_parking,open_sitting_available,resturant_tier,restaurant_type,restaurant_theme,restaurant_zomato_rating,restaurant_city_tier,order_wait_time,staff_responsivness,value_for_money,hygiene_rating,food_rating,overall_restaurant_rating,live_music_rating,comedy_gigs_rating,value_deals_rating,live_sports_rating,ambience,lively,service,comfortablility,privacy
0,60001,42000000,"indian,irish",Bangalore,Near Business Hub,14/02/09,84.3,Not Specific,95.8,1,1,1,0,1,2.0,Bar,Arabian,3,0,2,5,5,5,7,10.0,4.0,NaN,NaN,NaN,8.0,8,6,6,6


In [17]:
label_encode:list[pp.PreLabelEncoderConfig] = []
label_encode.append(pp.PreLabelEncoderConfig(
    column_name='city',
    label_encoding=list({pp.LabelEncodingDict(key, idx + 1) for idx, key in enumerate(train.city.value_counts().keys())}))
    )
label_encode.append(pp.PreLabelEncoderConfig(
    column_name='restaurant_location',
    label_encoding=list({pp.LabelEncodingDict(key, idx + 1) for idx, key in enumerate(train.restaurant_location.value_counts().keys())}))
    )
label_encode.append(pp.PreLabelEncoderConfig(
    column_name='endorsed_by',
    label_encoding=list({pp.LabelEncodingDict(key, idx + 1) for idx, key in enumerate(train.endorsed_by.value_counts().keys())}))
    )
label_encode.append(pp.PreLabelEncoderConfig(
    column_name='restaurant_theme',
    label_encoding=list({pp.LabelEncodingDict(key, idx + 1) for idx, key in enumerate(train.restaurant_theme.value_counts().keys())}))
    )
label_encode.append(pp.PreLabelEncoderConfig(
    column_name='restaurant_type',
    label_encoding=list({pp.LabelEncodingDict(key, idx + 1) for idx, key in enumerate(train.restaurant_type.value_counts().keys())}))
    )
label_encode.append(pp.PreLabelEncoderConfig(
    column_name='cuisine',
    label_encoding=list({pp.LabelEncodingDict(key, idx + 1) for idx, key in enumerate(train.cuisine.value_counts().keys())}))
    )

numeric_col_changer:list[pp.PreNumericColDataChangeConfig] = []
numeric_col_changer.append(pp.PreNumericColDataChangeConfig(col_name='facebook_popularity_quotient',data_type=int))
numeric_col_changer.append(pp.PreNumericColDataChangeConfig(col_name='instagram_popularity_quotient',data_type=int))
numeric_col_changer.append(pp.PreNumericColDataChangeConfig(col_name='overall_restaurant_rating',data_type=int))
numeric_col_changer.append(pp.PreNumericColDataChangeConfig(col_name='live_music_rating',data_type=int))
numeric_col_changer.append(pp.PreNumericColDataChangeConfig(col_name='ambience',data_type=int))
numeric_col_changer.append(pp.PreNumericColDataChangeConfig(col_name='resturant_tier',data_type=int))

model_config = pp.PreProcessingConfig(
    encoding_dummies=[],
    exclude_columns=['opening_day_of_restaurant','registration_number'],
    label_encode=label_encode,
    numeric_cols_data_changer=numeric_col_changer,
    target_column='annual_turnover'
)

In [18]:
pre_processed_df = pp.process(train,model_config)

Dropping columns whose values are nullable greater than 0.7
Removing unusable values like " " and "?"
Filling NA values with median and mode for numeria and non-numeric columns.
Performing IQR based outliers cleaning on target. annual_turnover
Changing datatype of numeric columns.
Performing label encoding on columns provided.
Encoding using dummies.


In [19]:
from auto_ml_kinder import model_training_data_prep as dp
from auto_ml_kinder import model_training_helper as mth
from auto_ml_kinder import model_list_helper as mlh

In [20]:
data = dp.ModelTrainingData(pre_processed_df, dp.ScalerType.QUANTILE_TRANSFORMER, .85,use_pca=False,use_polynomials=False,use_feature_selection=False)

Total columns being used after all data transformations: 28


In [21]:

trainer = mth.ModelTrainer(data=data)
trainer.perform_operation_regression(exclude_models=[
    # mlh.ModelAndParam.Linear_Regression
    # ,mlh.ModelAndParam.Ridge_Regression
    # ,mlh.ModelAndParam.Lasso_Regression
    # ,mlh.ModelAndParam.ElasticNet_Regression
    mlh.ModelAndParam.SVR_Regression
    ,mlh.ModelAndParam.DecisionTree_Regressor
    ,mlh.ModelAndParam.RandomForest_Regressor
    ,mlh.ModelAndParam.GradientBoosting_Regressor
    ,mlh.ModelAndParam.KNeighbors_Regressor

    ],permutate_n_less_column=0)
# trainer.performance_df.loc[trainer.performance_df['score'].idxmax()]

LinearRegression params: {'fit_intercept': [True, False]}
Total Permutations: 1
Ridge params: {'alpha': [0.1, 0.5, 1.0, 2.0, 5.0, 10.0, 20.0, 50.0, 100.0], 'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']}
Total Permutations: 1
Lasso params: {'alpha': [0.1, 0.5, 1.0, 2.0, 5.0, 10.0, 20.0, 50.0, 100.0], 'selection': ['cyclic', 'random']}
Total Permutations: 1
ElasticNet params: {'alpha': [0.1, 0.5, 1.0, 2.0, 5.0, 10.0, 20.0, 50.0, 100.0], 'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9], 'selection': ['cyclic', 'random']}
Total Permutations: 1


In [22]:
from auto_ml_kinder import neural_network_regression as nnr

In [39]:
trainer.perform_neural_network_regression(totalExperiments=4,
                                          params=mth.NeuralNetwork_BayesianOptimization_Params(
                                              neurons_min_max=(16,64)
                                              ,batch_size_min_max=(32,64)
                                              ,dropout_rate_min_max=(.3,.7)
                                              ,epochs_min_max=(100,150)
                                              ,hidden_layers_min_max=(1,3)
                                              ,learning_rate_min_max=(.01,.1)
                                              ,normalization_min_max=(0,1)
                                              ,dropout_min_max=(1,1)
                                              ))

|   iter    |  target   | activa... | batch_... |  dropout  | dropou... |  epochs   | hidden... | learni... |  neurons  | normal... | optimizer |
-------------------------------------------------------------------------------------------------------------------------------------------------
Neurons: 36||Total Columns:28||Normalization:False||Activation:elu||Total hidden layers:1||Dropout used:True||Dropout rate if used:0.6077049890092492||Optimizer:adadelta||Epochs:115||Batch_size:37

| 1         | -9.678e+1 | 5.51      | 37.41     | 1.0       | 0.6077    | 114.8     | 1.298     | 0.01202   | 36.17     | 0.2387    | 2.026     |
Neurons: 22||Total Columns:28||Normalization:False||Activation:relu||Total hidden layers:2||Dropout used:True||Dropout rate if used:0.567840095298652||Optimizer:nadam||Epochs:131||Batch_size:40

| 2         | -1.928e+1 | 8.916     | 39.61     | 1.0       | 0.5678    | 131.1     | 1.549     | 0.05196   | 21.68     | 0.07396   | 5.405     |
Neurons: 22||Total Colu

In [40]:
trainer.neural_network_best_model()

Neurons: 22
Total Columns:28
Normalization:False
Activation:relu
Total hidden layers:2
Dropout used:True
Dropout rate if used:0.567840095298652
Optimizer:nadam
Epochs:131
Batch_size:40

Epoch 1/131
48/48 [==============================] - 1s 4ms/step - loss: 967771022163968.0000 - val_loss: 937272258068480.0000
Epoch 2/131
48/48 [==============================] - 0s 3ms/step - loss: 967768337809408.0000 - val_loss: 937268164427776.0000
Epoch 3/131
48/48 [==============================] - 0s 2ms/step - loss: 967755855560704.0000 - val_loss: 937244407889920.0000
Epoch 4/131
48/48 [==============================] - 0s 2ms/step - loss: 967703376429056.0000 - val_loss: 937149717282816.0000
Epoch 5/131
48/48 [==============================] - 0s 3ms/step - loss: 967523927326720.0000 - val_loss: 936851015729152.0000
Epoch 6/131
48/48 [==============================] - 0s 3ms/step - loss: 967020745064448.0000 - val_loss: 936126843977728.0000
Epoch 7/131
48/48 [==============================] -

In [41]:
trainer.performance_df

,model_name,score,RMSE
0,Linear_Regression,0.233197,6.370587e+08
1,Ridge_Regression,0.231142,6.183848e+08
2,Lasso_Regression,0.233200,6.370519e+08
3,ElasticNet_Regression,0.201784,4.831488e+08
4,NeuralNetwork,-5.129244,3.180895e+07
5,NeuralNetwork,-0.247484,1.435038e+07
6,NeuralNetwork,-0.308504,1.469716e+07


In [27]:
test = pd.read_csv('../data/Test_dataset.csv')

In [28]:
test.columns = test.columns.str.lower().str.replace(' ', '_')

In [29]:
new_column_name = label_encode[2].column_name
test = test.rename(columns={'endoresed_by': new_column_name})

In [30]:
label_encode[0] = pp.PreLabelEncoderConfig(
    column_name='city',
    label_encoding=list({pp.LabelEncodingDict(key, idx + 1) for idx, key in enumerate(test.city.value_counts().keys())}))

In [31]:
model_config_test = pp.PreProcessingConfig(
    encoding_dummies=[],
    exclude_columns=['opening_day_of_restaurant','registration_number'],
    label_encode=label_encode,
    numeric_cols_data_changer=numeric_col_changer,
    target_column=''
)

In [32]:
test = pp.process_test(test,model_config_test)

Dropping columns whose values are nullable greater than 0.7
Removing unusable values like " " and "?"
Filling NA values with median and mode for numeria and non-numeric columns.
Changing datatype of numeric columns.
Performing label encoding on columns provided.
Encoding using dummies.


In [33]:
test.city.value_counts()

1     74
2     65
3     44
4     40
5     28
      ..
93     1
94     1
95     1
96     1
97     1
Name: city, Length: 97, dtype: int64